# This is based on Simple_postp_3.ipynb but with the following new function:
1)Merge branch to the newest pybamm-gem/develop
2) Use the exponential mesh refinement Simon used
3) Use the setting Simon uses for reproduce GEM results
4) Upgrade plot function to reduce script for plot
5) Include ageing cycle results.


In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing

In [4]:
ChemistryChen=pb.parameter_sets.OKane2022   
#ChemistryChen["electrolyte"] = "lipf6_Valoen2005"
Para_0=pb.ParameterValues(chemistry=ChemistryChen)
print(Para_0["Negative electrode LAM constant proportional term [s-1]"])
print(Para_0["Positive electrode LAM constant proportional term [s-1]"])
print(Para_0["Negative electrode diffusivity [m2.s-1]"])
print(Para_0["Positive electrode diffusivity [m2.s-1]"])
print(Para_0["SEI resistivity [Ohm.m]"])


2.7778e-07
2.7778e-07
<function graphite_LGM50_diffusivity_Chen2020 at 0x000001B19A036820>
4e-15
200000.0


In [5]:
print(Para_0['Outer SEI solvent diffusivity [m2.s-1]'])
print(Para_0['Bulk solvent concentration [mol.m-3]'])
print(Para_0["Dead lithium decay constant [s-1]"])
print(Para_0['Lithium plating kinetic rate constant [m.s-1]'])
print(Para_0['Positive electrode cracking rate'])
print(Para_0['Negative electrode cracking rate'])


2.5000000000000002e-22
2636.0
1e-06
1e-09
<function cracking_rate_Ai2020 at 0x000001B1BD7BCD30>
<function graphite_cracking_rate_Ai2020 at 0x000001B1BD7BC820>


In [3]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 

from Fun_P2_Crack_simp import (
    GetScan,
    recursive_scan,
    Para_init,
    Run_model_wwo_dry_out ,
    Cal_new_con_Update,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
    write_excel_xlsx,)

In [4]:
########################  Input  ########################
# all values here must be a list, even it is a single object
Para_dict_All = {
   "Total ageing cycles":[1092,],
   "Ageing cycles between RPT":[78,],
   "Update cycles for ageing":[13,],
   "Cycles within RPT":[2,],
   "Ageing temperature":[25,],
   "RPT temperature":[25,],
   "Particle mesh points":[120,],
   "Exponential mesh stretch":[2.3,],
   "Para_Set":[ "OKane2022",],
   "electrolyte":["lipf6_Nyman2008"],
   "Model option":[
         {
            "particle": "Fickian diffusion",          
            "SEI":"solvent-diffusion limited",   
            "SEI on cracks":"true",  
            "SEI film resistance":"distributed",          
            "SEI porosity change":"true",      
            "particle mechanics":("swelling and cracking", "swelling only"), 
            "loss of active material":"stress-driven", 
            "lithium plating":"partially reversible"      },
         ],
   # Solvent consumption sub-model
   "Initial electrolyte excessive amount ratio":[1.2,],
   "Current solvent concentration in the reservoir [mol.m-3]":[4541.0,],
   "Current electrolyte concentration in the reservoir [mol.m-3]":[1000,],
   "Ratio of Li-ion concentration change in electrolyte consider solvent consumption":[1.0,],
   # DFN parameter
   "Upper voltage cut-off [V]":[4.21,],
   "Lower voltage cut-off [V]":[2.49,],
   #"Negative electrode diffusivity [m2.s-1]":[2e-13,],
   #"Positive electrode diffusivity [m2.s-1]":[1e-13,],
   # general SEI
   'SEI resistivity [Ohm.m]':[2E5,],
   # solvent-diffusion limited
   'Outer SEI solvent diffusivity [m2.s-1]':[2.5e-22,],
   'Bulk solvent concentration [mol.m-3]':[4541.0,],
   # interstitial-diffusion limited
   'Inner SEI lithium interstitial diffusivity [m2.s-1]':[1e-12,],    
   'Lithium interstitial reference concentration [mol.m-3]':[15,],
   # ec-reaction limited
   'EC diffusivity [m2.s-1]':[2e-20,],
   'SEI kinetic rate constant [m.s-1]':[1e-19,],
   'EC initial concentration in electrolyte [mol.m-3]':[4541.0,],
   # LiP and coupling with SEI:
   "Dead lithium decay constant [s-1]":[ 1e-6,],
   'Lithium plating kinetic rate constant [m.s-1]':[1E-9,],
   # Crack model
   "Negative electrode LAM constant proportional term [s-1]":[ 
      2.7778e-7,],
   "Positive electrode LAM constant proportional term [s-1]":[ 2.7778e-7,],
   # make it simple for now,], but may want to have T dependency in the future
   "Negative electrode cracking rate":[ 3.9e-20,3.9e-21,],
   "Positive electrode cracking rate":[ 3.9e-20,],
   "Negative electrode volume change":[ 0.0,],
   "Positive electrode volume change":[ 0.0,],
   
}
Para_dict_list = []
recursive_scan(Para_dict_list,Para_dict_All, list(Para_dict_All.keys()), {})

# define experiments
V_max = 4.2;        
V_min = 2.5; 
exp_AGE_text = [(f"Discharge at 1 C until {V_min} V", 
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100"),  ]
# step index for ageing
step_AGE_CD =0;   step_AGE_CC =1;   step_AGE_CV =2;

exp_RPT_text = [ (f"Discharge at 0.1C until {V_min} V",  
        "Rest for 1 hours",  
        f"Charge at 0.1C until {V_max} V" ) ]
# step index for RPT
step_RPT_CD = 0;  step_RPT_RE =1;   step_RPT_CC = 2;  

exp_text_list = [exp_AGE_text, exp_RPT_text,];
cycle_no = -1; 
exp_index_pack = [cycle_no,step_AGE_CD,step_AGE_CC,step_AGE_CV,
   step_RPT_CD,step_RPT_RE , step_RPT_CC ];



# Test - write in excel
type(str(model_options))
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P2R3/'; 
Target  = 'Test_crack_xx/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target);
book_name_xlsx = 'write_head.xlsx';

sheet_name_xlsx = 'Results';

Path_pack = [BasicPath,Target,book_name_xlsx,sheet_name_xlsx,];


value3 = [
   ["Index", "Ratio_ex","cs_Neg_Init", "Diff_SEI", "R_SEI", 
   "Bulk_Sol_Con","D_Li_inSEI", "c_Li_inte_ref",
   "Diff_EC","k_SEI","LAMcr_prop","Crack_rate",
   "Couple_SEI_LiP","k_LiP","Temper_i","mesh_par",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final","Error",str(model_options)],
   ]
write_excel_xlsx(BasicPath + Target+book_name_xlsx, sheet_name_xlsx, value3)


In [5]:
########################  Output  ########################
keys_loc_RPT = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode roughness ratio",
    "CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    "CDend Negative electrode roughness ratio",
    "CDend Total SEI on cracks thickness [m]",
    "REend Total SEI on cracks thickness [m]",
]
keys_tim_RPT = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "RE Terminal voltage [V]",
]
keys_cyc_RPT = [   # default: CDend
    "Discharge capacity [A.h]",
    "CDend Loss of capacity to lithium plating [A.h]",
    "CDend Loss of capacity to SEI [A.h]",
    "CDend Loss of capacity to SEI on cracks [A.h]",
    "CDend X-averaged total SEI on cracks thickness [m]",
    "CDend X-averaged negative electrode roughness ratio",
    "CDend Local ECM resistance [Ohm]",
    "CDsta Negative electrode SOC", 
    "CDend Negative electrode SOC",
    "CDsta Positive electrode SOC", 
    "CDend Positive electrode SOC",
    "CDend Negative electrode capacity [A.h]",
    "CDend Positive electrode capacity [A.h]",
]

keys_loc_AGE = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode roughness ratio",
    "CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    "CDend Negative electrode roughness ratio",
    "CDend Total SEI on cracks thickness [m]",
]
keys_tim_AGE = [];
keys_cyc_AGE = [];
keys_all_RPT = [keys_loc_RPT,keys_tim_RPT,keys_cyc_RPT];
keys_all_AGE = [keys_loc_AGE,keys_tim_AGE,keys_cyc_AGE];
keys_all = [keys_all_RPT,keys_all_AGE];

In [6]:
# Main code to run the model
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P2R3/'; 
Target  = 'Reproduce_GEM_Office/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target);
book_name_xlsx = 'Solv_SEI_1092cycles_1.xlsx';

sheet_name_xlsx = 'Results';

Path_pack = [BasicPath,Target,book_name_xlsx,sheet_name_xlsx,];

# Write the head for excel file:
head_list = list(Para_dict_list[0].keys())
head_list.insert(0,"Index");
head_list.insert(1,"Dry out");
head_list.extend(["Model option", "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final","Error"])
write_excel_xlsx(BasicPath + Target+book_name_xlsx, sheet_name_xlsx, [head_list])


# scan:
index_xlsx = 0; DryOut = "Off"; # option: On and Off
for Para_dict_i in Para_dict_list:
   index_xlsx += 1;
   Run_model_wwo_dry_out(
      index_xlsx, Para_dict_i,   Path_pack , 
      keys_all,   exp_text_list, exp_index_pack, DryOut )



Successfully create a excel file
Finish the break-in cycle
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1
Solve up to Step 0
Solve up to Step 1


ValueError: Cannot convert {'particle': 'Fickian diffusion', 'SEI': 'solvent-diffusion limited', 'SEI on cracks': 'true', 'SEI film resistance': 'distributed', 'SEI porosity change': 'true', 'particle mechanics': ('swelling and cracking', 'swelling only'), 'loss of active material': 'stress-driven', 'lithium plating': 'partially reversible'} to Excel

In [22]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P2R3/'; 
Target  = 'Reproduce_GEM_Office/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target);
book_name_xlsx = 'Test_write_model_option.xlsx';

sheet_name_xlsx = 'Results';

Path_pack = [BasicPath,Target,book_name_xlsx,sheet_name_xlsx,];

model_option = {
            "particle": "Fickian diffusion",          
            "SEI":"solvent-diffusion limited",   
            "SEI on cracks":"true",  
            "SEI film resistance":"distributed",          
            "SEI porosity change":"true",      
            "particle mechanics":("swelling and cracking", "swelling only"), 
            "loss of active material":"stress-driven", 
            "lithium plating":"partially reversible"      }
str_model_option = str(model_option)
str_str = str(str_model_option)
print(type(str_model_option))
print(type(model_option))

print(type(str_str))
print(str_str)

<class 'str'>
<class 'dict'>
<class 'str'>
{'particle': 'Fickian diffusion', 'SEI': 'solvent-diffusion limited', 'SEI on cracks': 'true', 'SEI film resistance': 'distributed', 'SEI porosity change': 'true', 'particle mechanics': ('swelling and cracking', 'swelling only'), 'loss of active material': 'stress-driven', 'lithium plating': 'partially reversible'}


In [11]:
# Write the head for excel file:
head_list = list(Para_dict_list[0].keys())
head_list.insert(0,"Index");
head_list.insert(1,"Dry out");
head_list.extend(["Model option", "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final",
   "Error",str_model_option])
write_excel_xlsx(BasicPath + Target+book_name_xlsx, sheet_name_xlsx, [head_list])


Successfully create a excel file


In [21]:
data = openpyxl.load_workbook(BasicPath + Target+ book_name_xlsx)         
table = data.get_sheet_by_name('Results')
nrows = table.max_row  # 获得行数
values = list(Para_dict_list[0].keys())
values.insert(0,"Index");
values.insert(1,"Dry out");
values.extend([str_model_option,"Model option", "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final",
   "Error",])
values = [values,]
for i in range(1, len(values)+1):
    for j in range(1, len(values[i-1])+1):
        table.cell(nrows+i, j).value = str(values[i-1][j-1])    
data.save(BasicPath + Target+ book_name_xlsx)

In [19]:
print(values[0][-1])
print(type(values[0][0]))

Error
<class 'str'>
